# Masked Language Modeling

## Importing the libraries

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

## Defining the model name and tokenizer

In [ ]:
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

## Creating the mask

In [ ]:
mask = tokenizer.mask_token
print(mask)

sentence = f"How many people can you fit inside of a {mask}?"

tokens = tokenizer.tokenize(sentence)

encoded_inputs = tokenizer(sentence, return_tensors="pt")

outputs = model(**encoded_inputs)

logits = outputs.logits.detach().numpy()[0]
# logits are like the score that the model gives each word on its ability to fit into the mask
print(logits.shape)
# The first number is the number of tokens in the string (8 plus start and end), the second is the total vocabulary size of the model

## Predict the most likely token to replace the masked token

In [ ]:
mask_logits = logits[tokens.index(mask) + 1]
confidence_scores = softmax(mask_logits)

for i in np.argsort(confidence_scores)[::-1][:5]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]
    print(sentence.replace(mask, pred_token), score)